Задача 1. Профиль пользователя (валидация полей)
Постройте модель профиля пользователя для внутренней CRM:

Требования

Обязательные поля: id: UUID, email: EmailStr, name: str.
Опциональные поля: website: HttpUrl | None, bio: str | None.
Пароль хранится как SecretStr, должен быть не короче 8 символов.
Имя (name) нормализуйте: тримминг + одна пробельная последовательность между словами + первая буква каждого слова заглавная.
Если указан website, домен сайта не должен совпадать с доменом email (смысл: личный сайт != корпоративная почта).
Подсказки: используйте field_validator для нормализации и локальных проверок; и model_validator(mode="after") для проверки зависимости email ↔ website.

In [25]:
from typing import Optional
from pydantic import BaseModel, Field, EmailStr, HttpUrl, SecretStr
from pydantic import field_validator, model_validator
from uuid import UUID, uuid4
import logging
import os
import sys



def setup_logging():
    
    """Настройка системы логирования"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('app.log', encoding='utf-8'),
            logging.StreamHandler(sys.stdout)
        ]
    )
    
    logging.getLogger('pydantic').setLevel(logging.WARNING)
    logging.getLogger('sqlalchemy').setLevel(logging.WARNING)

setup_logging()

logger = logging.getLogger(__name__)

class UserProfile(BaseModel):
    
    id: UUID = Field(default_factory=uuid4)
    email: EmailStr
    name: str
    password: SecretStr
    website: Optional[HttpUrl] = None
    bio: Optional[str] = None


    @field_validator("name")
    @classmethod
    def normalize_name(cls, v: str) -> str:
        
        """Нормализация имени: тримминг + один пробел между словами + заглавные буквы"""
        
        if not v.strip():
            raise ValueError("Имя не может быть пустым")
            
        original = v
        
        v = v.strip()
        v = ' '.join(v.split())
        v = v.title()
        
        logger.info(f"Нормализовано имя: '{original}' -> '{v}'")
        return v

 
    @field_validator("password")
    @classmethod
    def password_strength(cls, v: SecretStr) -> SecretStr:
        
        """Проверка длины пароля"""
        
        password = v.get_secret_value()
        if len(password) < 8:
            raise ValueError("Пароль должен содержать не менее 8 символов")
            
        logger.debug("Пароль прошел валидацию длины")
            
        return v


    @model_validator(mode="after")
    def check_domains(self):
        
        """Проверка, что домен website не совпадает с доменом email"""
        
        if self.website is not None:
            
            email_domain = self.email.split('@')[1].lower()
            website_domain = urlparse(str(self.website)).hostname.lower()
            
            if website_domain.startswith('www.'):
                website_domain = website_domain[4:]
            
            if website_domain == email_domain:
                
                logger.warning(
                    f"Конфликт доменов: email={email_domain}, website={website_domain}"
                )
                
                raise ValueError(
                    f"Домен личного сайта ({website_domain}) не должен совпадать "
                    f"с доменом корпоративной почты ({email_domain})"
                )
                
        logger.info(f"Создан профиль пользователя: {self.email}")
        
        return self

Задача 2. Валидация функции заказа (@validate_call)
Реализуйте функцию place_order, которая принимает:

user_id: UUID
sku: str (артикул, только заглавные буквы/цифры, длина 3–12)
quantity: int (>0)
price: Decimal (>= 0), округляется банковским методом до 2 знаков
Функция должна возвращать словарь с ключами: user_id, sku, quantity, price, amount (quantity × price).

Используйте @validate_call и локальные проверки через обычный код (или вспомогательные валидаторы TypeAdapter не используем).

In [15]:
from pydantic import validate_call
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
import re


@validate_call
def place_order(user_id, sku, quantity, price):
    
    logger.info(f"Создание заказа: user={user_id}, sku={sku}, qty={quantity}")
    
    """Функция валидации данных заказа"""
    
    if not re.match(r'^[A-Z0-9]{3,12}$', sku):
        
        logger.warning(f"Неверный формат SKU: {sku}")
        raise ValueError("SKU должен содержать только заглавные буквы и цифры, длина 3-12 символов")
        
    if quantity <= 0:
        logger.warning(f"Неверное количество: {quantity}")
        raise ValueError("Количество должен быть больше 0")
        
    if price < 0:
        logger.warning(f"Отрицательная цена: {price}")
        raise ValueError("Цена не может быть отрицательным")
        
    rounded_price = price.quantize(Decimal('0.01'), rounding=ROUND_HALF_EVEN)
    
    amount = quantity * rounded_price
    
    logger.info(f"Заказ создан: user={user_id}, total={amount}")
    
    return {
        "user_id": user_id,
        "sku": sku,
        "quantity": quantity,
        "price": rounded_price,
        "amount": amount
    }

Задача 3. Модель заказа с бизнес-правилами
Смоделируйте заказ в магазине цифровых товаров.

Требования

OrderStatus: Enum со значениями new, paid, delivered, canceled.
Модель OrderItem:
sku: str как в задаче 2
qty: int (>0)
unit_price: Decimal (>=0) округление до 2 знаков
Модель Order:
id: UUID
user_email: EmailStr
items: list[OrderItem] (не пустой)
status: OrderStatus = 'new'
created_at: datetime (по умолчанию datetime.utcnow)
Расчитанное поле total: Decimal — сумма по всем позициям
В model_validator(mode="after") запретите переход в paid/delivered при total == 0 и запретите пустые корзины.
Важно: используйте только инструменты pydantic и стандартную библиотеку.

In [16]:
from pydantic import BaseModel, EmailStr, field_validator, model_validator
from typing import List
from decimal import Decimal, ROUND_HALF_EVEN
from uuid import UUID
from datetime import datetime
from enum import Enum

SKU_RE = re.compile(r"^[A-Z0-9]{3,12}$")

class OrderStatus(str, Enum):
    
    NEW = "new"
    PAID = "paid"
    DELIVERED = "delivered"
    CANCELED = "canceled"

class OrderItem(BaseModel):
    sku: str
    qty: int
    unit_price: Decimal

    @field_validator("sku")
    @classmethod
    def sku_format(cls, v: str) -> str:
        
        """Функция валидации SKU-номера заказа"""
        
        if not SKU_RE.match(v):
            logger.warning(f"Неверный формат SKU: {v}")
            raise ValueError("SKU должен содержать только заглавные буквы и цифры, длина 3-12 символов")
        return v

    @field_validator("qty")
    @classmethod
    def qty_positive(cls, v: int) -> int:
        
        """Функция валидации количества товаров в заказе"""
        
        if v <= 0:
            logger.warning(f"Неверное количество товара: {v}")
            raise ValueError("Количество должно быть больше 0")
        return v

    @field_validator("unit_price")
    @classmethod
    def price_non_negative(cls, v: Decimal) -> Decimal:
        
        """Функция валидации цены товаров в заказе"""
        
        if v < 0:
            raise ValueError("Цена не может быть отрицательной")
        return v.quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)
    
    

class Order(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    user_email: EmailStr
    items: List[OrderItem]
    status: OrderStatus = OrderStatus.NEW
    created_at: datetime = Field(default_factory=datetime.utcnow)
        
    @property
    def total(self) -> Decimal:
        
        """Расчетное поле: общая сумма заказа"""
        
        total = sum(item.qty * item.unit_price for item in self.items)
        logger.debug(f"Рассчитана сумма заказа {self.id}: {total}")
        return total
    

    @model_validator(mode="after")
    def check_business_rules(self):
    
        """Бизнес-правила"""
        
        if not self.items:
            logger.warning(f"Пустая корзина в заказе {self.id}")
            raise ValueError("Корзина не может быть пустой")
            
        if self.status in [OrderStatus.PAID, OrderStatus.DELIVERED] and self.total == 0:
            logger.warning(
                f"Попытка оплаты бесплатного заказа {self.id}, статус={self.status}"
            )
            raise ValueError(
                f"Нельзя перевести заказ с нулевой суммой в статус '{self.status.value}'"
            )
        
        logger.info(f"Заказ {self.id} успешно создан: total={self.total}, items={len(self.items)}")
        
        return self
    

Задача 4. Конфигурация приложения (BaseSettings)
Опишите настройки подключения к внешнему API:

APISettings(BaseSettings) с полями:
base_url: HttpUrl
token: SecretStr
timeout_sec: int = 5 (1–60)
retries: int = 2 (0–10)
Используйте model_config = ConfigDict(env_prefix="API_", env_file=".env", extra="ignore")
Проверьте, что значения корректно читаются из переменных окружения.
В тесте ниже среда заполняется вручную.

In [17]:
from pydantic_settings import BaseSettings
from pydantic import ConfigDict, SecretStr, HttpUrl, field_validator
import unittest.mock

class APISettings(BaseSettings):
    
    base_url: HttpUrl
    token: SecretStr
    timeout_sec: int = 5
    retries: int = 2

        
    @field_validator("timeout_sec", "retries")
    @classmethod
    def check_ranges(cls, v: int, info):
        
        """Проверка значений "timeout_sec", "retries" """
        
        field_name = info.field_name
        
        if field_name == "timeout_sec":
            if not 1 <= v <= 60:
                logger.error(f"Неверный timeout_sec: {v}, должен быть 1-60")
                raise ValueError(f"timeout_sec должен быть в диапазоне 1-60, получено: {v}")
        
        elif field_name == "retries":
            if not 0 <= v <= 10:
                logger.error(f"Неверный retries: {v}, должен быть 0-10")
                raise ValueError(f"retries должен быть в диапазоне 0-10, получено: {v}")
        
        return v

    model_config = ConfigDict(
        
        env_prefix="API_",
        env_file=".env",
        extra="ignore"
        
    )
    
    
def test_api_settings():
    
    """Тест с временным изменением окружения"""
    
    with unittest.mock.patch.dict(os.environ, {
        "API_BASE_URL": "https://test-api.com/v1/",
        "API_TOKEN": "test-token-123",
        "API_TIMEOUT_SEC": "15",
        "API_RETRIES": "3"
    }):
        settings = APISettings()
        
        assert str(settings.base_url) == "https://test-api.com/v1/", f"Ожидалось 'https://test-api.com/v1/', получено '{str(settings.base_url)}'"
        assert settings.token.get_secret_value() == "test-token-123"
        assert settings.timeout_sec == 15
        assert settings.retries == 3
    
    print("Успешно! APISettings корректно читает настройки из окружения.")

if __name__ == "__main__":
    test_api_settings()

Успешно! APISettings корректно читает настройки из окружения.


Задача 5. Извлечение из ORM (from_attributes=True)
Создайте простую SQLAlchemy-модель SAUser(id, email, is_active) (in-memory, без БД) и соответствующую модель Pydantic:

Pydantic-модель UserOut с полями id: UUID, email: EmailStr, is_active: bool.
Включите поддержку from_attributes в model_config.
Создайте инстанс SAUser и провалидируйте его через UserOut.model_validate(sa_user_instance).
Проверьте, что преобразование сработало.

In [18]:
from typing import Optional
from sqlalchemy import Column, String, Boolean
from sqlalchemy.orm import declarative_base
from uuid import uuid4
from pydantic import BaseModel, EmailStr, ConfigDict

Base = declarative_base()

class SAUser(Base):
    __tablename__ = "users"
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    email = Column(String, nullable=False)
    is_active = Column(Boolean, default=True)

    def __init__(self, email: str, is_active: bool = True):
        self.id = str(uuid4())
        self.email = email
        self.is_active = is_active
        
        logger.debug(f"Создан SAUser: {self.email}, active={self.is_active}")

class UserOut(BaseModel):
    
    id: UUID
    email: EmailStr
    is_active: bool
    
    model_config = ConfigDict(
        
        from_attributes=True
        
    )
    

def test_orm_conversion():
    
    """Проверка преобразования из SQLAlchemy модели в Pydantic модель"""
    
    sa_user = SAUser(
        email="user@example.com",
        is_active=True
    )
    
    print("SQLAlchemy объект:")
    print(f"  id: {sa_user.id} (тип: {type(sa_user.id)})")
    print(f"  email: {sa_user.email} (тип: {type(sa_user.email)})")
    print(f"  is_active: {sa_user.is_active} (тип: {type(sa_user.is_active)})")
    
    user_out = UserOut.model_validate(sa_user)
    
    print("\nPydantic объект:")
    print(f"  id: {user_out.id} (тип: {type(user_out.id)})")
    print(f"  email: {user_out.email} (тип: {type(user_out.email)})")
    print(f"  is_active: {user_out.is_active} (тип: {type(user_out.is_active)})")
    
    assert str(user_out.id) == sa_user.id
    assert user_out.email == sa_user.email
    assert user_out.is_active == sa_user.is_active
    
    print("\nПреобразование работает корректно!")
    return user_out

if __name__ == "__main__":
    result = test_orm_conversion()    


SQLAlchemy объект:
  id: 123f37e8-a92a-46bb-94e7-28a71454673f (тип: <class 'str'>)
  email: user@example.com (тип: <class 'str'>)
  is_active: True (тип: <class 'bool'>)

Pydantic объект:
  id: 123f37e8-a92a-46bb-94e7-28a71454673f (тип: <class 'uuid.UUID'>)
  email: user@example.com (тип: <class 'str'>)
  is_active: True (тип: <class 'bool'>)

Преобразование работает корректно!


Задача 6. JSON Schema и дружелюбные ошибки
Для модели из задачи 3 сгенерируйте JSON Schema (метод model_json_schema) и запишите его в переменную ORDER_SCHEMA.
Реализуйте функцию safe_create_order(data: dict) -> tuple[bool, str], которая:
пытается создать Order из входного dict,
при успехе возвращает (True, "<total=...>"),
при ошибке возвращает (False, "<короткое сообщение об ошибке>") без стек-трейса.
Не используйте сторонние библиотеки.

In [21]:
from typing import Tuple 
from decimal import Decimal
import json

# Используем модели из задачи 3: OrderStatus, OrderItem, Order

try:
    ORDER_SCHEMA = Order.model_json_schema()
    logger.info("JSON Schema для Order успешно сгенерирована")
except Exception as e:
    logger.error(f"Ошибка генерации JSON Schema: {str(e)}")
    raise

def safe_create_order(data: dict) -> Tuple[bool, str]:
    
    """Создание заказа"""
    
    logger.info(f"Попытка создания заказа: email={data.get('user_email')}, items={len(data.get('items', []))}")
    
    try:
        order = Order(**data)
        logger.info(f"Заказ успешно создан: id={order.id}, total={order.total}")
        return True, f"Заказ создан! total={order.total}"
        
    except Exception as e:
        error_msg = str(e)
        logger.warning(f"Ошибка валидации заказа: {error_msg}")
        
        if "email" in error_msg.lower():
            return False, "Неверный формат email"
        elif "sku" in error_msg.lower():
            return False, "Неверный формат артикула"
        elif "количество" in error_msg.lower():
            return False, "Количество должно быть больше 0"
        elif "корзина" in error_msg.lower() or "пустой" in error_msg.lower()::
            return False, "Корзина не может быть пустой"
        elif "нулевой суммой" in error_msg.lower():
            return False, "Нельзя оплатить бесплатный заказ"
        else:
            return False, f"Ошибка данных: {error_msg}"
            
    except Exception as e:
        logger.error(f"Критическая ошибка при создании заказа: {str(e)}", exc_info=True)
        return False, "Внутренняя ошибка сервера"

    

SyntaxError: invalid syntax (<ipython-input-21-a20719a0ff47>, line 35)

Тестирование

In [31]:
def test_app():
    
    """Тестирование основных функций приложения"""
    
    try:
        user = UserProfile(
            name="иван Иванов",
            email="ivan@example.com", 
            password="password123"
        )
        print(f"Пользователь создан: {user.email}")
    except Exception as e:
        print(f"Ошибка: {e}")
    
    try:
        order_data = {
            "user_email": "ivan@example.com",
            "items": [
                {"sku": "PRODUCT1", "qty": 2, "unit_price": Decimal("25.50")},
                {"sku": "ITEM2", "qty": 1, "unit_price": Decimal("10.00")}
            ]
        }
        
        success, message = safe_create_order(order_data)
        if success:
            print(f"{message}")
        else:
            print(f"Ошибка при создании заказа: {message}")
            
    except Exception as e:
        print(f"Ошибка: {e}")
    
    try:
        
        result = place_order(
            user_id=uuid4(),
            sku="PRODUCT2",
            quantity=1,
            price=Decimal("49.99")
        )
        print(f"Заказ через place_order: {result['amount']}")
    except Exception as e:
        print(f"Ошибка: {e}")
    
    
    # Неверный email
    try:
        bad_order = {
            "user_email": "email",
            "items": [{"sku": "PRODUCT3", "qty": 1, "unit_price": Decimal("10.00")}]
        }
        success, message = safe_create_order(bad_order)
        print(f"Ошибка по email: {message}")
    except Exception as e:
        print(f"Другая ошибка: {e}")
    
    # Пустая корзина
    try:
        empty_order = {
            "user_email": "test@test.com",
            "items": []
        }
        success, message = safe_create_order(empty_order)
        print(f"Ошибка пустой корзины: {message}")
    except Exception as e:
        print(f"Другая ошибка: {e}")
    
    print("\nТест завершен")

def test_json_schema():
    
    """Тест JSON схемы"""
    
    try:
        schema = Order.model_json_schema()
        print(f"\nСхема создана")
        print(f"   Поля: {list(schema['properties'].keys())}")
    except Exception as e:
        print(f"Ошибка схемы: {e}")
        
    print("\nТест завершен")

# Запуск тестов
if __name__ == "__main__":
    test_app()
    test_json_schema()

2025-11-23 18:25:55,332 - __main__ - INFO - Нормализовано имя: 'иван Иванов' -> 'Иван Иванов'
2025-11-23 18:25:55,334 - __main__ - INFO - Создан профиль пользователя: ivan@example.com
Пользователь создан: ivan@example.com
2025-11-23 18:25:55,336 - __main__ - INFO - Попытка создания заказа: email=ivan@example.com, items=2
2025-11-23 18:25:55,349 - __main__ - INFO - Заказ 04ce6013-5e4f-4796-afa9-0fb6e32264d5 успешно создан: total=61.00, items=2
2025-11-23 18:25:55,363 - __main__ - INFO - Заказ успешно создан: id=04ce6013-5e4f-4796-afa9-0fb6e32264d5, total=61.00
Заказ создан! total=61.00
2025-11-23 18:25:55,366 - __main__ - INFO - Создание заказа: user=ef3233a5-496d-4b82-ba93-9e2906ea585b, sku=PRODUCT2, qty=1
2025-11-23 18:25:55,368 - __main__ - INFO - Заказ создан: user=ef3233a5-496d-4b82-ba93-9e2906ea585b, total=49.99
Заказ через place_order: 49.99
2025-11-23 18:25:55,372 - __main__ - INFO - Попытка создания заказа: email=email, items=1
2025-11-23 18:25:55,387 - __main__ - WARNING - Оши